<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2022-2/Insumos/Merton_KMV_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
import yfinance as yf


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 631 kB/s 
     |████████████████████████████████| 6.4 MB 10.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
start = '2020-01-01'

symbol = 'WALMEX.MX'
stock_data = yf.Ticker(symbol)
# stock_data = stock_data.history(start=start)#, end=end) 


In [3]:
stock_data.info

{'52WeekChange': 0.06348717,
 'SandP52WeekChange': -0.029111743,
 'address1': 'Nextengo #78',
 'address2': 'Santa Cruz Acayucan',
 'algorithm': None,
 'annualHoldingsTurnover': None,
 'annualReportExpenseRatio': None,
 'ask': 68.8,
 'askSize': 470000,
 'averageDailyVolume10Day': 17604677,
 'averageVolume': 17416640,
 'averageVolume10days': 17604677,
 'beta': 0.144347,
 'beta3Year': None,
 'bid': 68.52,
 'bidSize': 50100,
 'bookValue': 10.958,
 'category': None,
 'circulatingSupply': None,
 'city': 'Mexico City',
 'companyOfficers': [],
 'country': 'Mexico',
 'currency': 'MXN',
 'currentPrice': 68.6,
 'currentRatio': 1.161,
 'dateShortInterest': None,
 'dayHigh': 69.51,
 'dayLow': 68.3,
 'debtToEquity': 34.118,
 'dividendRate': 1.9,
 'dividendYield': 0.0277,
 'earningsGrowth': 0.103,
 'earningsQuarterlyGrowth': 0.103,
 'ebitda': 78373167104,
 'ebitdaMargins': 0.10418,
 'enterpriseToEbitda': 15.837,
 'enterpriseToRevenue': 1.65,
 'enterpriseValue': 1241182240768,
 'exDividendDate': 16686

In [22]:
Acc=stock_data.info["sharesOutstanding"]
Kp=stock_data.info["enterpriseValue"]
Dp=stock_data.info["totalDebt"]
# print(stock_data.balance_sheet)
import pandas as pd
bs=pd.DataFrame(stock_data.balance_sheet)
K_0=bs[["2021-12-31"]].loc["Total Stockholder Equity"]
# bs["Total Stockholder Equity"]
D_0=bs[["2021-12-31"]].loc["Total Liab"]


In [24]:
print(Kp, "vs", K)
print(Kp/K)
print(Dp, "vs", D)
print(Dp/D)

1241182240768 vs 
2021-12-31    1.858820e+11
Name: Total Stockholder Equity, dtype: float64

2021-12-31    6.677259
Name: Total Stockholder Equity, dtype: float64
65279737856 vs 
2021-12-31    2.085075e+11
Name: Total Liab, dtype: float64

2021-12-31    0.313081
Name: Total Liab, dtype: float64


In [26]:
import pandas as pd
import numpy as np
from statistics import stdev

df = pd.DataFrame()

hist=stock_data.history(period="max", auto_adjust=True)

df['ds'] = hist.index
df['y'] = hist['Close'].values

df.tail()

,ds,y
4914,2022-06-03,71.190002
4915,2022-06-06,69.269997
4916,2022-06-07,68.500000
4917,2022-06-08,69.349998
4918,2022-06-09,68.599998


In [41]:
Delta_A = df[df.ds<="2021-12-31"]['y'].pct_change(1) # 1 for ONE DAY lookback
vol_A=stdev(Delta_A[Delta_A.notnull()])*np.sqrt(252)

vol_A

0.26430736486136486

In [51]:
A_0=df[df.ds=="2021-12-31"]['y']
A_0= A_0*Acc
print(A)


print(A/Kp)

print(K)
print(A/np.array(K))
# print(K/A)


4808    1.328638e+12
Name: y, dtype: float64
4808    1.070462
Name: y, dtype: float64

2021-12-31    1.858820e+11
Name: Total Stockholder Equity, dtype: float64
4808    7.147749
Name: y, dtype: float64


In [ ]:
from scipy.optimize import fsolve
from scipy.stats import norm
norm.ppf(.5)
x=c(D_0,vol_A)

#Función para estimar vol_k y D
def G(x):
  vol_k = x[2]
  D = x[1]
  F[0]=A_0-K_0*norm.ppf((np.log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
  A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
  


G = function(x) 
{
  vol_k <- x[2]
  D <- x[1]
  c(A_0-K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))+D*exp(-r*T)*pnorm((log(K_0/D)+(r-0.5*vol_k^2)*T)/(vol_k*sqrt(T))),
  A_0*vol_A-vol_k*K_0*pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T))) )
}

#K_0=164370

ss=multiroot(G,x,positive = TRUE) #RESOLVER EL SISTEMA.

D=ss$root[1]
vol_k=ss$root[2]
ss$root[1]/D_0
vol_A/ss$root[2]
G(c(D,vol_k))

#Probabilidad de incumplimiento
PD_merton=1-pnorm((log(K_0/D)+(r+0.5*vol_k^2)*T)/(vol_k*sqrt(T)))
PD_merton